In [2]:
import asyncio
import websockets
import json
import time

import sys
sys.path.append("./chainklik")
import pandas as pd
import requests
from web3 import Web3

import libs.common.utils as utils
import libs.common.payload as payload
import config.config as cfg

In [3]:
# Connect to an Ethereum node
w3 = Web3(Web3.HTTPProvider(cfg.config["eth_sepolia_http_url"]))

# Set sender and recipient addresses
sender_address = '0x5bA4D4264Bf9A8C3aaF7e1fea6f83f50643A3Fd7'
recipient_address = '0xaf6667a2F847beeca6a6604126Dc28344518840b'

# Set private key for the sender's account. 
private_key = '1dab201501e8b882ca3413edcdfed263e4834cd8ea4c9586aca7fb699c51d681'

In [47]:
w3.eth.get_block("latest")["number"]

5007928

In [48]:
bt = BlockTimer(w3.eth.get_block("latest")["number"], w3.eth.get_block("latest")["number"] + 100, 5)
data_node = DataNode("block_data", {}, {}, bt)

In [ ]:
tx_node = TxNode("tx",{},{},)

In [2]:
async def process_message(block_info):
    print(block_info["number"])
    ctx = {
        "block_time" : int(block_info["number"], 16)
    }
    values = {}
    data_node.run(ctx, values)

In [3]:
async def message():
    async with websockets.connect(cfg.config["eth_sepolia_ws_url"]) as ws:
        sub_newheads = { "id":1, "jsonrpc":"2.0", "method":"eth_subscribe", "params":["newHeads"] }
        await ws.send(json.dumps(sub_newheads))
        sub_res = await ws.recv()
        print(json.loads(sub_res))
        while True:
            message = await asyncio.wait_for(ws.recv(), timeout=60)
            block_info = json.loads(message)["params"]["result"]
            # start = time.time()
            await process_message(block_info)
            # end = time.time()
            # print(end-start)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

loop = asyncio.get_event_loop()
while True:
    loop.run_until_complete(message())

{'jsonrpc': '2.0', 'id': 1, 'result': '0xec7238b8fd206cbc3bd1ecf29bb415ab'}
0x4c6a22
0.00018787384033203125
0x4c6a23
0.00019097328186035156


In [4]:
balance_sender = w3.from_wei(w3.eth.get_balance(sender_address), 'ether')
balance_recipient = w3.from_wei(w3.eth.get_balance(recipient_address), 'ether')

print(f'The balance of { sender_address } is: { balance_sender } ETH')
print(f'The balance of { recipient_address } is: { balance_recipient } ETH')

The balance of 0x5bA4D4264Bf9A8C3aaF7e1fea6f83f50643A3Fd7 is: 0.489531614690461 ETH
The balance of 0xaf6667a2F847beeca6a6604126Dc28344518840b is: 0.01 ETH


In [5]:
from abc import ABC, abstractmethod

class Timer(ABC):
    def __init__(self, start, end, frequency):
        self.start = start
        self.end = end
        self.frequency = frequency
        assert self.start < self.end

    @abstractmethod
    def is_active(self, ctx):
        pass

    @abstractmethod
    def is_expired(self, ctx):
        pass
        
class BlockTimer(Timer):
    def __init__(self, start, end, frequency):
        super().__init__(start, end, frequency)

    def is_active(self, ctx):
        if ctx["block_time"] - self.start >= 0:
            self.start = ((ctx["block_time"] - self.start) // self.frequency + 1) * self.frequency + self.start
            return True
        else:
            return False

    def is_expired(self, ctx):
        if self.start >= self.end:
            return True
        return False

In [6]:
class TimerDecorator:
    def __init__(self, timer_attr):
        self.timer_attr = timer_attr

    def __call__(self, func):
        def wrapper(*args, **kwargs):
            timer = getattr(args[0], self.timer_attr)
            ctx = args[1]
            if not timer.is_expired(ctx) and timer.is_active(ctx):
                func(*args, **kwargs)
        return wrapper

In [7]:
class Node(ABC):
    def __init__(self, id: str, deps: dict, params: dict):
        self.id = id
        self.deps = deps
        self.params = params
        self.output = None
        self.active = False
        self.finalized = True

    @abstractmethod
    def run(self, ctx: dict, values: dict) -> None:
        pass

In [8]:
class DataNode(Node):
    def __init__(self, id: str, deps: dict, params: dict, timer: Timer = None):
        super().__init__(id, deps, params)
        self.timer = timer
    
    @TimerDecorator("timer")
    def run(self, ctx: dict, values: dict) -> None:
        self.output = values
        self.active = True
        self.finalized = True

In [49]:
class TxNode:
    def __init__(self, id: str, deps: dict, params: dict):
        super().__init__(id, deps, params)
        self.wallet = params["wallet"]
        self.to = params["to"]
        self.amount = params["amount"]
        self.finality = params["finality"] if "finality" in params else 2
        self.gas = params["gas"] if "gas" in params else 2
    
    def run(self, ctx: dict, values: dict):
        if self.finalized: # a new or finalized txn
            latest_block = w3.eth.get_block("latest")
            base_fee_per_gas = latest_block.baseFeePerGas   # Base fee in the latest block (in wei)
            max_priority_fee_per_gas = w3.to_wei(1, 'gwei') # Priority fee to include the transaction in the block
            max_fee_per_gas = (5 * base_fee_per_gas) + max_priority_fee_per_gas # Maximum amount you’re willing to pay 
            
            transaction_params = {
                'from': self.wallet,
                'to': self.to,
                'value': w3.to_wei(self.amount, 'ether'),
                'nonce': w3.eth.get_transaction_count(self.wallet),
                'gas': self.gas, 
                'maxFeePerGas': max_fee_per_gas, # Maximum amount you’re willing to pay 
                'maxPriorityFeePerGas': max_priority_fee_per_gas, # Priority fee to include the transaction in the block
                'chainId': 11155111 # ChainId of Sepolia Testnet
            }
        
            transaction = w3.eth.account.sign_transaction(transaction_params, private_key)
            transaction_hash = w3.eth.send_raw_transaction(transaction.rawTransaction)
            transaction_receipt = w3.eth.wait_for_transaction_receipt(transaction_hash)
        
            if transaction_receipt.status:
                print('Transaction successful!')
                print('Transaction hash:', transaction_hash.hex())
                print(f'Explorer link: https://sepolia.etherscan.io/tx/{transaction_hash.hex()}')
                self.tx_block = transaction_receipt["blockNumber"]
                self.status = "unfinalized"
            else:
                print('Transaction failed.')
                self.status = "failed"
        elif ctx.block_time - self.tx_block >= self.finality:
            self.finalized = True